In [1]:
# 在Colab上首先安装依赖库
!pip install ultralytics pyyaml

# 挂载Google Drive（如果需要从Drive读取数据）
from google.colab import drive
drive.mount('/content/drive')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 978.8/978.8 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 84.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 62.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 31.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [2]:
!mkdir -p /content/trained_models

In [4]:
import os

# 检查关键路径是否存在
required_paths = [
    "/content/drive/MyDrive/Project/YOLOv8犬种类别检测/YOLO_Dataset/Train_YOLO/images",
    "/content/drive/MyDrive/Project/YOLOv8犬种类别检测/YOLO_Dataset/Validation_YOLO/images",
    "/content/drive/MyDrive/Project/YOLOv8犬种类别检测/YOLO_Dataset/data.yaml"
]

for path in required_paths:
    if not os.path.exists(path):
        raise FileNotFoundError(f"关键路径不存在: {path}")
    else:
        print(f"✅ 路径验证通过: {path}")

✅ 路径验证通过: /content/drive/MyDrive/Project/YOLOv8犬种类别检测/YOLO_Dataset/Train_YOLO/images
✅ 路径验证通过: /content/drive/MyDrive/Project/YOLOv8犬种类别检测/YOLO_Dataset/Validation_YOLO/images
✅ 路径验证通过: /content/drive/MyDrive/Project/YOLOv8犬种类别检测/YOLO_Dataset/data.yaml


In [5]:
from ultralytics.yolo.utils import ops

# 在训练前执行数据集验证
ops.check_dataset(data_yaml_path)

ModuleNotFoundError: No module named 'ultralytics.yolo'

In [6]:
from ultralytics import YOLO
import os

# 确保Colab能访问Google Drive（如果尚未挂载）
from google.colab import drive
drive.mount('/content/drive')

# 设置路径变量
base_path = "/content/drive/MyDrive/Project/YOLOv8犬种类别检测"
# 确保data.yaml路径正确指向Colab环境
data_yaml_path = "/content/drive/MyDrive/Project/YOLOv8犬种类别检测/YOLO_Dataset/data.yaml"
pretrained_model = os.path.join(base_path, "pre_trained_model/yolov8n.pt")

# 加载预训练模型
model = YOLO(pretrained_model)

# 训练参数配置
train_params = {
    'data': data_yaml_path,
    'epochs': 200,
    'batch': 16,         # 根据GPU内存调整（T4建议16，V100可尝试32）
    'imgsz': 640,
    'device': 'cuda',    # 使用GPU加速
    'optimizer': 'Adam', # 或'SGD'（YOLO默认）
    'lr0': 0.01,         # 初始学习率（通常比Adam更保守的值）
    'lrf': 0.01,         # 最终学习率（lr0 * lrf）
    'augment': True,     # 自动启用基础数据增强
    'mosaic': 0.75,      # 启用马赛克增强的概率
    'hsv_h': 0.015,     # 色调增强幅度
    'hsv_s': 0.7,       # 饱和度增强幅度
    'hsv_v': 0.4,       # 明度增强幅度
    'translate': 0.1,   # 平移增强幅度
    'scale': 0.5,       # 缩放增强幅度
    'fliplr': 0.5,      # 水平翻转概率
    'workers': 2,       # Colab推荐值
    'project': os.path.join(base_path, "trained_models"),
    'name': 'dog_classification_v1',
    'exist_ok': True,   # 允许覆盖同名实验
    'seed': 42          # 固定随机种子
}

# 开始训练
results = model.train(**train_params)

# 自动验证最佳模型（使用验证集）
metrics = model.val()  # 会自动使用data.yaml中的验证集路径

# 测试集评估（需要data.yaml中已配置test集路径）
if os.path.exists(os.path.join(base_path, "YOLO_Dataset/Test_YOLO")):
    test_results = model.val(
        split='test',
        save_json=True,  # 生成JSON格式评估结果
        save_hybrid=True # 保存混合标签以分析误检
    )

# 预测示例
test_image = os.path.join(base_path, "YOLO_Dataset/Test_YOLO/images/test.jpg")
if os.path.exists(test_image):
    prediction = model.predict(
        test_image,
        conf=0.5,        # 置信度阈值
        iou=0.45,        # NMS IoU阈值
        save=True,
        save_txt=True,   # 保存标签结果
        save_conf=True,  # 标签中保存置信度
        exist_ok=True
    )

# 模型导出（保存到Google Drive）
export_dir = os.path.join(base_path, "exported_models")
os.makedirs(export_dir, exist_ok=True)

# 导出ONNX格式
model.export(
    format='onnx',
    imgsz=640,
    dynamic=False,  # 固定输入尺寸（Colab部署推荐）
    simplify=True,  # 简化ONNX模型
    opset=12,       # ONNX版本
    file=os.path.join(export_dir, 'dog_classifier.onnx')
)

# 保存PyTorch格式
model.save(os.path.join(export_dir, 'dog_classifier.pt'))

print("训练流程完成！")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Ultralytics 8.3.111 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=/content/drive/MyDrive/Project/YOLOv8犬种类别检测/pre_trained_model/yolov8n.pt, data=/content/drive/MyDrive/Project/YOLOv8犬种类别检测/YOLO_Dataset/data.yaml, epochs=200, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cuda, workers=2, project=/content/drive/MyDrive/Project/YOLOv8犬种类别检测/trained_models, name=dog_classification_v1, exist_ok=True, pretrained=True, optimizer=Adam, verbose=True, seed=42, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False,

100%|██████████| 755k/755k [00:00<00:00, 21.7MB/s]


Overriding model.yaml nc=80 with nc=23

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytic

100%|██████████| 5.35M/5.35M [00:00<00:00, 95.4MB/s]


AMP: checks passed ✅
train: WARNING ⚠️ Slow image access detected (ping: 10.8±22.8 ms, read: 0.1±0.1 MB/s, size: 50.5 KB). Use local storage instead of remote/mounted storage for better performance. See https://docs.ultralytics.com/guides/model-training-tips/


...

val: Scanning /content/drive/MyDrive/Project/YOLOv8犬种类别检测/YOLO_Dataset/Test_YOLO/labels... 420 images, 0 backgrounds, 0 corrupt: 100%|██████████| 420/420 [02:33<00:00,  2.73it/s]

val: New cache created: /content/drive/MyDrive/Project/YOLOv8犬种类别检测/YOLO_Dataset/Test_YOLO/labels.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 27/27 [00:09<00:00,  2.77it/s]


                   all        420        450      0.741      0.634      0.752      0.628
                 husky         34         36        0.6      0.666      0.679      0.593
                beagle         19         21       0.53      0.476      0.549      0.458
             chihuahua         15         16      0.473       0.25      0.362      0.267
              pekinese         15         15      0.828      0.533      0.747      0.672
              shih_tzu         21         22      0.824      0.455      0.755      0.608
          afghan_hound         24         26      0.829       0.93      0.933       0.85
              airedale         20         20      0.797        0.9      0.895      0.809
                basset         18         18      0.658      0.722      0.748      0.624
      blenheim_spaniel         19         23      0.758      0.478      0.686       0.51
            bloodhound         19         20       0.67      0.711       0.73      0.601
                 corg

SyntaxError: '[31m[1mfile[0m' is not a valid YOLO argument. Similar arguments are i.e. ['profile=False'].

    Arguments received: ['yolo', '-f', '/root/.local/share/jupyter/runtime/kernel-516b578c-edfd-439a-8798-950ffdf71375.json']. Ultralytics 'yolo' commands use the following syntax:

        yolo TASK MODE ARGS

        Where   TASK (optional) is one of frozenset({'segment', 'pose', 'obb', 'classify', 'detect'})
                MODE (required) is one of frozenset({'val', 'benchmark', 'track', 'predict', 'train', 'export'})
                ARGS (optional) are any number of custom 'arg=value' pairs like 'imgsz=320' that override defaults.
                    See all ARGS at https://docs.ultralytics.com/usage/cfg or with 'yolo cfg'

    1. Train a detection model for 10 epochs with an initial learning_rate of 0.01
        yolo train data=coco8.yaml model=yolo11n.pt epochs=10 lr0=0.01

    2. Predict a YouTube video using a pretrained segmentation model at image size 320:
        yo